# AdAwareness

---



---


Clone https://github.com/irisong980912/gender_bias

# Import Datasets

## Youtube8M videos

Follow the instructions from here https://github.com/gsssrao/youtube-8m-videos-frames

  » Download youtube-dl and ffmpeg
  
  » Create a txt file selectedcategories.txt with 'Television advertisement' in the first line and a second blank line.
  
  » bash downloadmulticategoryvideos.sh 0 selectedcategories.txt
  
  ## Pittsburg videos
  
  Follow instructions in the github  Readme

  # Extract frames
  
 bash extract_frames.sh 
 
 (you might need to change the path) 


https://github.com/irisong980912/gender_bias/blob/master/extract_frames.sh

# DeepAI Nudity Detector

python3 nudity.py -» Google Drive

# Algorithmia 

In [0]:
import time

import platform
import io

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [0]:
import torchvision.models as models

import os
from google.colab import files
import numpy as np
import pandas as pd

In [0]:
# TODO: read the image label input
  # - maybe need to change image file format? (ask Jean-Philip)

In [0]:
# TODO: perform the 

###Video title preprocessing


1.   Bags of  words / Tokenizer?
2.   Sequence padding?
3.   Word2vec or an embedding layer before the lstm layer?






In [0]:
def prepare_sequence():
  pass

##DATASET PREPROCESSING

In [0]:
import os
import math
import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np
import torch.utils.data
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from torchvision.transforms import ToTensor

from torch.utils.data.sampler import SubsetRandomSampler

In [0]:
use_gpu = torch.cuda.is_available()

In [0]:
def make_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)

def separate_train_test(dataset_path, train_path, test_path, test_split=0.2, extension = ".png"):
    class_index = 1
    for classname in sorted(os.listdir(dataset_path)):
        make_dir(os.path.join(train_path, classname))
        make_dir(os.path.join(test_path, classname))
        i = 0
        for file in sorted(os.listdir(os.path.join(dataset_path, classname))):
            if file.endswith(extension):
                continue
            classname_path = os.path.join(dataset_path, classname)
            file_path = os.path.join(dataset_path, classname, file)
            if i < len(os.listdir(classname_path)) * test_split:
                copyfile(file_path, os.path.join(test_path, classname, file))
            else:
                copyfile(file_path, os.path.join(train_path, classname, file))
            i += 1

        class_index += 1

#TODO

def attribute_labels(datset_path):
  pass
      
  
def train_valid_loaders(dataset, batch_size, train_split=0.8, shuffle=True):
    num_data = len(dataset)
    indices = np.arange(num_data)

    if shuffle:
        np.random.shuffle(indices)

    split = math.floor(train_split * num_data)
    train_idx, valid_idx = indices[:split], indices[split:]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(dataset,
                                               batch_size=batch_size, sampler=train_sampler)
   
    valid_loader = torch.utils.data.DataLoader(dataset,
                                               batch_size=batch_size, sampler=valid_sampler)

    return train_loader, valid_loader
  
  
def train_test_valid_loaders(train_path, test_path, train_split, transformer):

    train_dt = ImageFolder(root=train_path, transform=transformer)
    test_dt = ImageFolder(root=test_path, transform=transformer)
    train_loader, val_loader = train_valid_loaders(train_dt, batch_size=batch_size, train_split=train_split)
    test_loader, _= train_valid_loaders(dataset, batch_size, train_split=1)
    
    return train_loader, test_loader, valid_loader


In [0]:
dataset_path=
train_path=
test_path =  #TODO
test_split=
transformer = 

separate_train_test(dataset_path, train_path, test_path, test_split=test_split)

attribute_labels(datset_path)

train_loader, test_loader, valid_loader = train_test_valid_loaders(train_path, test_path, train_split, transformer)



In [0]:
class History:

    def __init__(self):
        self.history = {
            'train_acc': [],
            'val_acc': [],
            'train_loss': [],
            'val_loss': []
        }

    def save(self, train_acc, val_acc, train_loss, val_loss):
        self.history['train_acc'].append(train_acc)
        self.history['val_acc'].append(val_acc)
        self.history['train_loss'].append(train_loss)
        self.history['val_loss'].append(val_loss)

    def display(self, save):
        epoch = len(self.history['train_acc'])
        epochs = [x for x in range(1, epoch + 1)]

        fig, axes = plt.subplots(2, 1)
        plt.tight_layout()

        axes[0].set_title('Train accuracy')
        axes[0].set_xlabel('Epochs')
        axes[0].set_ylabel('Accuracy')
        axes[0].plot(epochs, self.history['train_acc'], label='Train')
        axes[0].plot(epochs, self.history['val_acc'], label='Validation')
        axes[0].legend()

        axes[1].set_title('Train loss')
        axes[1].set_xlabel('Epochs')
        axes[1].set_ylabel('Loss')
        axes[1].plot(epochs, self.history['train_loss'], label='Train')
        axes[1].plot(epochs, self.history['val_loss'], label='Validation')
        plt.savefig(save)
        plt.show()
        
  
def validate(model, val_loader, use_gpu=False):
    true = []
    pred = []
    val_loss = []

    criterion = nn.CrossEntropyLoss()
    model.eval()

    with torch.no_grad():
        for j, batch in enumerate(val_loader):

            inputs, targets = batch
            if use_gpu:
                inputs = inputs.cuda()
                targets = targets.cuda()

            output = model(inputs)

            predictions = output.max(dim=1)[1]

            val_loss.append(criterion(output, targets).item())
            true.extend(targets.data.cpu().numpy().tolist())
            pred.extend(predictions.data.cpu().numpy().tolist())

    return accuracy_score(true, pred) * 100, sum(val_loss) / len(val_loss)


def train(model,train_loader, val_loader, n_epoch, batch_size, learning_rate, loss_fn, use_gpu=False):
    history = History()

    criterion = loss_fn
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    for i in range(n_epoch):
        model.train()
        for j, batch in enumerate(train_loader):

            inputs, targets = batch
            if use_gpu:
                inputs = inputs.cuda()
                targets = targets.cuda()

            optimizer.zero_grad()
            output = model(inputs)

            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()

        train_acc, train_loss = validate(model, train_loader, use_gpu)
        val_acc, val_loss = validate(model, val_loader, use_gpu)
        history.save(train_acc, val_acc, train_loss, val_loss)
        print('Epoch {} - Train acc: {:.2f} - Val acc: {:.2f} - Train loss: {:.4f} - Val loss: {:.4f}'.format(i,
                                                                                                              train_acc,
                                                                                                              val_acc,
                                                                                                              train_loss,
                                                                                                              val_loss))
    return history    
    
def test(model, test_loader,use_gpu=False):
    loader = test_loader
    return validate(model, loader, use_gpu)

def save(model, f):
   torch.save(model, f)

def load(f):
    model = torch.load(f, map_location='cpu')
    model.eval()
    return model

In [0]:
num_classes = 
n_epoch = 
batch_size = 
learning_rate = 
model_path =

##LINEAR REGRESSION MODEL

In [0]:
class NUDNet():
    
    def __init__(self, input_dim, hidden_sizes, output_dim, b_norm=False, dropout=0.0):
        super(NUDNet, self).__init__()
        layers = []
        in_ = input_dim
        for out_ in hidden_sizes:
            layers.append(nn.Linear(in_, out_))
            layers.append(nn.Dropout(p=dropout))
            if b_norm:
                layers.append(nn.BatchNorm1d(out_))
            layers.append(nn.ReLU())

            in_ = out_
            
        layers.append(nn.Linear(in_, output_dim))
        self.net = nn.Sequential(*layers)


    def forward(self, inputs):
        return self.net(inputs)

In [0]:

loss_fn = nn.CrossEntropyLoss()
input_dim = 
hidden_sizes = 
output_dim = 1

net = NUDNet(inp)
if use_gpu:
    net = net.cuda()

history = train(model=net, train_loader, valid_loader , learning_rate=learning_rate, n_epoch=n_epoch,
                    use_gpu=use_gpu, loss_fn=loss_fn)
   
acc, loss = test(model=net, test_loader, batch_size=batch_size, use_gpu=use_gpu)
    
save(net, model_path )




loss_func = torch.nn.MSELoss()

## CNN
Convolutional neural networks model to process key frames labeled with Deep AI Nudity Detection and Skin Detection

###VGNET

In [0]:
class VGNet():
    def __init__(self, num_classes):
        super(VGNet, self).__init__()
        self.model = models.vgg19(pretrained=True)
        for param in self.model.parameters():
            param.requires_grad = False
        self.model.classifier[6] = nn.Linear(self.model.classifier[6].in_features, num_classes)
        
    def forward(x):
      # TO DO : X need to be specified. Is it a video (many frames)? I will be back here.
      return self.model(x)

###LOAD DATASET

###TRAINING AND TESTING

In [0]:
num_classes = 
n_epoch = 
batch_size = 
learning_rate = 
model_path =
loss_fn = nn.CrossEntropyLoss()

net = VGNet(num_classes)

if use_gpu:
    net = net.cuda()

history = train(model=net, train_loader, valid_loader , learning_rate=learning_rate, n_epoch=n_epoch,
                    use_gpu=use_gpu, loss_fn=loss_fn)
   
acc, loss = test(model=net, test_loader, batch_size=batch_size, use_gpu=use_gpu)
    
save(net, model_path )


## LSTM
Long short-term memory model to process titles of the videos associated with the key frames

In [0]:
class LSTM_model(nn.Module):    
    def __init__(self, input_size, hidden_size, 
                 num_layers=1, dropout=0, bidirectional=False):
        super(LSTM_model, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        self.lstm = nn.LSTM(input_size,
                            hidden_size,
                            num_layers,
                            dropout=dropout,
                            bidirectional=bidirectional)
        self.linear = nn.Linear(hidden_size, 1)
        
    def forward(self, inputs, hidden): 
        outputs, hidden = self.lstm(inputs, hidden)
        predictions = self.linear(outputs[-1])
        return predictions.squeeze(1), outputs, hidden
    
    def init_hidden(self, batch_size):
        num_directions = 2 if self.bidirectional else 1
        hidden = (torch.zeros(self.num_layers*num_directions, batch_size, self.hidden_size),
                  torch.zeros(self.num_layers*num_directions, batch_size, self.hidden_size))
        return hidden

## Multimodal Deep Learning Model
- CNN + LSTM + Linear model
- Input: video (frames), title of the video
- Output: prediction of male or female nudity labeled with context

In [0]:
class multimodal(nn.Module):
  def __init__(self, frame, title, num_classes):
    super(multimodal, self).__init__()
    
    # CNN model to procee frames
    self.model_1 = PreNet(num_classes)

    # LSTM model to process title
    self.model_2 = 
    
    # Linear model to predict nudity
    self.model_3 = nn.Sequential(
       nn.Linear(100352, 10)
    )

    
  def forward(self, x, t):
    # TODO: loop through the video x to get frames
    out1 = self.model_1(frame)
    
    out2 = self.model_2(t)
    
    # Combine outputs from CNN and LSTM models
    # out =
    
    # Predict male or female nudity
    out = self.model_3(out)